# **Automatic Summarization**

Algortimo semplice:
1. individua argomento del testo come una lista di vettori di Nasari (termine1, score1, termine2, score2, ...)
2. crea contesto raccogliendo vettori dei termini trovati al passo prima. Nel caso in cui il titolo sia troppo corto (e quindi poco informativo) posso cercare vettori di elementi che fanno parte delle definizioni dei termini trovati al passo 1
3. peso i paragrafi in base alla somma dei pesi dei termini che fanno parte del contesto con la Weighted Overlap. Mantengo solo i paragrafi che hanno un peso maggiore di una soglia

Valutazione:
- BLEU
- ROUGE

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

### Funzioni di utility per il preprocessing

In [19]:
# Remove stopwords and punctuation from the text, tokenize it and lemmatize it
def preprocess(text):
    text = text.lower()
    stop = []
    with open('/Users/jak/Documents/Uni/TLN/TLN/Radicioni/data/stop_words_FULL.txt', 'r') as f:
        stop = f.read().splitlines()
    stop = set(stop)
    punct = ['.', ',', '!', '?', ':', ';', '(', ')', '[', ']', '{', '}', '"', "'", '``', "''", '...', '’', '“', '”']
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop and t not in punct]
    lemmatizer = WordNetLemmatizer()
    tokens = set([lemmatizer.lemmatize(t) for t in tokens])
    return [t for t in tokens if not t.startswith(punct) and not t.endswith(punct)]
    # return set([lemmatizer.lemmatize(t) for t in tokens])
    # tokens = set([lemmatizer.lemmatize(t) for t in tokens])
    # tokens = [t for t in tokens if t not in stop and t not in punct]
    # return tokens

# try preprocessing

preprocess("This is a test. It's a test of the preprocessing system.")


20


TypeError: startswith first arg must be str or a tuple of str, not list